In [ ]:
import numpy as np 
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM , Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import pandas as pd

In [ ]:
df = pd.read_csv('D:/My WorkSpace/Summer-Internship/23-July-2024/flipkart.csv')
df.head(2)

In [ ]:
def label_rating(Rating):
    if Rating >=4:
        return 2 # represents positive
    elif Rating ==3:
        return 1 # neutral
    else:
        return 0 # negative

In [ ]:
df['labels']=df['Rating'].apply(label_rating)
df

In [ ]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df['Review'])
sequences = tokenizer.texts_to_sequences(df['Review'])
data = pad_sequences(sequences,maxlen=200)


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data, df['labels'], test_size=0.2, random_state=42)

In [ ]:
y_train=to_categorical(y_train,num_classes=3)
y_test=to_categorical(y_test,num_classes=3)


In [ ]:
model = Sequential()
model.add(Embedding(input_dim=5000,output_dim=128,input_length=200))
model.add(LSTM(units=128,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(units=3,activation='softmax'))


model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(x_train, y_train, epochs=10,batch_size=32,validation_split=0.2)

loss, accuracy = model.evaluate(x_test, y_test)

print(f"Loss is {loss} and Accuracy is {accuracy}")

In [ ]:
def predict_user_input(text):
    sequences=tokenizer.texts_to_sequences([text])
    padded_seq = pad_sequences(sequences,maxlen=200)
    prediction=model.predict(padded_seq)
    label = prediction.argmax(axis=1)[0]
    if label==2:
        return "Customer Satisfied"
    elif label==1:
        return "Customer Partially Satisfied"
    else:
        return "Customer Not Satisfied"
    

user_review=input("Enter your review please : ")
result = predict_user_input(user_review)
print(result)